In [370]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet as wn
from nltk.probability import FreqDist
from tqdm import tqdm
import re
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import spacy
import pypandoc

# increase limit of characters processed
# need to specify to not use spcifics models to avoid overload
#nlp_es.max_length = 2000000

## (fonctions)

In [175]:
# funct. import text, clean withite spaces, return cleaned text as list
def clean_input(txt_file):
    file_content = open(txt_path + file + ".txt", "r").readlines()
    file_cleaned = []
    for line in file_content:
        if not line.isspace():
            file_cleaned.append(line.strip('\n'))
    return(file_cleaned)




## data description


| Acronym      | Source | Langage     |
| :---         |    :----:   |          ---: |
|TS_DIC  |dictionnary | Tikuna-Spanish |
|LLB_NEW |New Testament La Ligua Biblica |Tikuna|
|LLB_OLD |Old Testament La Ligua Biblica |Tikuna|
|T_Ebible|Ebible |Tikuna|
|S_Ebible|EBible| Spanish|
|T_OHCR  |Universal Declaration Of Human Rights | Tikuna|
|S_OHCR  |Universal Declaration Of Human Rights  |Spanish|
|T_CRU  |Crubadan Synsets  |Tikuna|
|T_CANT  |readaloud bible parts  |Tikuna|

- Ebible not converted in TXT for now
- Missing Spanish llb bible


In [176]:
# Create dictionnary with all the content
content = {}
# import text files
name_files_txt = ['llb_new','llb_old','t_ohcr','s_ohcr','t_cru']
txt_path = 'SOURCES/txt/'
for file in name_files_txt :
    file_content = open(txt_path + file + ".txt", "r").readlines()
    file_cleaned = []
    for line in file_content:
        if not line.isspace():
            file_cleaned.append(line.strip('\n'))
    content[file] = file_cleaned


# TODO : instead of length os text, transform into number of tokens and add csv

In [202]:
# Create dictionary to store content
content2 = {}
# Define files names
# TODO get from directory
name_files_txt = ['llb_new','llb_old','t_ohcr','s_ohcr','t_cru']
# explicit path
# TODO clean name
txt_path = 'SOURCES/txt/'
# Fill the doctionary with the clean content
for file in name_files_txt:
    content2[file] = clean_input(file)

In [203]:
# clean to get only functions
def fill_dic(file_list):
    dic_content = {}
    for file in file_list:
        dic_content[file] = clean_input(file)
    return(dic_content)

s = fill_dic(name_files_txt)

In [249]:
# Len of text corpus
token_size_texts = {}
for key,value in content2.items():
    text = ''.join(value)
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tok_words = tokenizer.tokenize(text)
    n_tokens = len(tok_words)
    token_size_texts[key] = n_tokens
token_size_texts = pd.DataFrame(token_size_texts, index=[0])
token_size_texts
# Remove punctuation to have only words


,llb_new,llb_old,t_ohcr,s_ohcr,t_cru
0,293437,276768,2582,1888,3035


### Plots

In [179]:
# Plot size of text files
length_dict = {key: len(value) for key, value in content.items()}
df_size = pd.DataFrame(length_dict.items(),columns=['name','size'])
df_size = pd.pivot_table(df_size,index='name')
# plot_size = df_size.plot(kind='bar');
# TODO : adapt axis, sort sizes, legends, title

### Tikuna - Spanish dictionnary
#### todo:
- rename columns
- table token_pos
- token_spa list > spacy tag
- compare ?
- Create new columns
- examples spa, exemple tik
- pos tag examples


In [189]:
# Import new output from Anderson dictionary
ts_dic = pd.read_csv('SOURCES/csv/ts_dic.csv')
# ts_dic




In [192]:
example = []
for i in range(ts_dic.shape[0]):
    str = ts_dic['dic_entry'][i]
    match = re.search(r'([A-Z].*\.)',str)
    if match:
        str2 = match.group(0)
        match2 = re.search(r'[\(]',str2)
        if match2:
            example.append('')
        else:
            example.append(match.group(0))
    else:
        example.append('')
    # * cases, tikuna esp
    # mismatch an entry without pos tag
    # Spanish descritpion of meaning
ts_dic['example'] = example

exclude = [None,'',' ','Var.','\n','\t','\r','\f']
s = 0
for i in range(ts_dic.shape[0]):
    str = ts_dic['example'][i]
    match = re.search(r'(?P<tikuna>[A-Z].*(\.|\!|\?|\¿))\s*'
    '(?P<spanish>[A-ZÈÉ].*(\.|\!|\?|\¿))',str)
    if match:
        #print(match.group(1))
        pass
    else:
        if ts_dic['example'][i] not in exclude:
            str = ts_dic['example'][i]
            match = re.search(r'[A-Z].*\.?\s?[A-Z].*\.?',str)
            if match:
                #print(ts_dic['example'][i])
                match2 = re.search(r'(Indica|Se usa|\[])',str)
                if match2:
                    pass
                else:
                    ts_dic['example'][i] = re.sub(r'(\s+([A-Z]))',r'\.\1',str)
            else:
                #print(ts_dic['example'][i])
                s += 1
                pass
print(s)
# in the 220 examples, still good example with tikuna and spanish
# Spanish begins with lowe case, difficult match and add point.
# Verify number of line and clean manually or check with a spanish dict.
# Embed this in a fonction



In [196]:
# Descriptive data on the dictinonary
#ts_dic
# Count : number of exmaples, number of missing spanish (def)
# Look for "missing occurwnce acroos all the dataset"
# Longueur des exemmpes
# Compter le nombre de mot unique en esp
# how many spanis word appearing many time in entries
# install the python wrapper, check how many occurences already in the spanish worndet
#ts_dic['token_pos'].value_counts()
# Split list of pos for categories

#ts_dic['token_spa'].isnull().sum()
#len(ts_dic[ts_dic['example'] != '']) 




In [ ]:
# ebible text processing
# import files
# sum in one
# dictionary with file name numbers and content
# find pattern between spanish and ticuna in names

Spanish tikuna bibles
LLB : find selected tikuna and match part with spanigh
ebible:
all in one text
align put in single dictionary



In [334]:
directory = 'SOURCES/txt/spa_ebible'
spa_ebible = {}
master_f = []

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        if filename.endswith(".txt"):
            rename = re.search(r'([0-9]{3,3}.*[0-9]{2,2})', filename)
            content = open(os.path.join(directory, filename), "rt").read()  
            master_f.append(content)          
            spa_ebible[rename.group(0)] = content

spa_ebible =  dict(sorted(spa_ebible.items()))

In [335]:
directory = 'SOURCES/txt/tik_ebible'
tik_ebible = {}
master_f = []

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        if filename.endswith(".txt"):
            rename = re.search(r'([0-9]{3,3}.*[0-9]{2,2})', filename)
            content = open(os.path.join(directory, filename), "rt",encoding="UTF-8-sig").read()
            content = re.sub('\n','',content)
            master_f.append(content)          
            tik_ebible[rename.group(0)] = content

tik_ebible =  dict(sorted(tik_ebible.items()))

In [374]:
directory = 'SOURCES/original/tcaNT_html'
tik_ebible_full = {}

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        if filename.endswith(".htm"):
            output = pypandoc.convert_file(f, 'rst',format = 'html')
            filename = re.sub('\.htm','',filename)
            tik_ebible_full[filename] = output




In [376]:
directory = 'SOURCES/original/spavbl_html'
spa_ebible_full = {}

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        if filename.endswith(".htm"):
            output = pypandoc.convert_file(f, 'rst',format = 'html')
            filename = re.sub('\.htm','',filename)
            spa_ebible_full[filename] = output

# compare loading html and cleaning with 
pypandoc.convert_file('chapters/*.md', 'docx', outputfile="somefile.docx")
and create text files
then match names of texts in tikuna and spanish
create a graphical map in the notebook


In [355]:
# install pyandoc

##  spacy

### Load models

In [184]:
# load large spacy models
nlp_es = spacy.load("es_dep_news_trf")




### pos tagging texts in spanish

In [200]:
# spacy tag function. list as input, list of tupple as output.
def tag_text(text_list):
    tag_list = []
    s = ''.join([str(line) for line in text_list])
    doc = nlp_es(s,disable=['ner'])
    for token in doc:
        token_tupple = (token.text, token.pos_, token.dep_)
        tag_list.append(token_tupple)
    return(tag_list)



In [ ]:
# Annoter les données en html espagnol - tikuna pour aligner
# txt, paragraphe, phrase, token
# Delta Crubadan & Dataset
# Match dictionnary content with bible
# Create function for matching efficient